### Select the best pretrained model

Check the benchamarks of models here and choose the best model: https://pytorch.org/docs/stable/torchvision/models.html

In [1]:
import torchvision

In [3]:
model = torchvision.models.wide_resnet101_2(pretrained=True, progress=False)

Downloading: "https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth" to /Users/atiyatarkash/.cache/torch/checkpoints/wide_resnet101_2-32ee1156.pth


### Load data and analyse it

In [4]:
import os

In [5]:
os.listdir("./")

['.DS_Store',
 'challenges-in-representation-learning-facial-expression-recognition-challenge',
 'README.md',
 '.ipynb_checkpoints',
 '.git',
 'Model.ipynb',
 '.idea']

In [6]:
data_dir = "challenges-in-representation-learning-facial-expression-recognition-challenge"

In [7]:
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
os.listdir(data_dir)

In [ ]:
tar = tarfile.open(data_dir + "/fer2013.tar.gz")
df = pd.read_csv(tar.extractfile("fer2013/fer2013.csv"))
df.head()

In [ ]:
df["Usage"].value_counts()

In [ ]:
train = df[["emotion", "pixels"]][df["Usage"] == "Training"]
train.head()

In [ ]:
train['pixels'] = train['pixels'].apply(lambda im: np.fromstring(im, sep=' '))
x_train = np.vstack(train['pixels'].values)
y_train = np.array(train["emotion"])
x_train.shape, y_train.shape

In [ ]:
test_df = df[["emotion", "pixels"]][df["Usage"]=="PublicTest"]

In [ ]:
test_df["pixels"] = test_df["pixels"].apply(lambda im: np.fromstring(im, sep=' '))
x_test = np.vstack(test_df["pixels"].values)
y_test = np.array(test_df["emotion"])

In [ ]:
x_train = x_train.reshape(-1, 48, 48, 1)
x_test = x_test.reshape(-1, 48, 48, 1)
x_train.shape, x_test.shape

In [ ]:
import seaborn as sns
plt.figure(0, figsize=(12,6))
for i in range(1, 13):
    plt.subplot(3,4,i)
    plt.imshow(x_train[i, :, :, 0], cmap="gray")

plt.tight_layout()
plt.show()

### Create dataloaders

In [ ]:
class NumpyDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):

        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return (image, label)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

trainset = NumpyDataset(images=x_train, labels=y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = NumpyDataset(images=x_test, labels=y_test)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

In [ ]:
classes = []

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

### Create loss and optimizer

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Train the model

In [ ]:
epochs = 10

In [ ]:
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

### Save model

In [ ]:
torch.save(net.state_dict(), PATH)

### Test

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))